<a href="https://colab.research.google.com/github/higor-gomes93/employee_attrition/blob/main/Employee_Attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Completo - Employee Attrition**

#### **Problema**
Iremos realizar uma análise completa de dados envolvendo o problema de predição de saída de colaboradores em uma empresa. Os dados utilizados podem ser encontrados [aqui](https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset). A anáise será dividida em quatro etapas:
1. Análise básica: conexão com a base de dados, limpeza completa do dataset e análise exploratória gráfica
2. Análise intermediária: correlações, testes de hipóteses e análise de sobrevivência
3. Análise avançada: modelos preditivos, feature importance, oversampling e escolha de um modelo
4. Análise expert: feature selection, grid search e avaliação das métricas de sucesso nas bases de treino e teste com random search

#### **Objetivos**
Identificar quais são as principais alavancas que estão levando as pessoas deixarem a empresa, validar algumas hipóteses e desenvolver um modelo preditivo para essas saídas.




---



### 1) Instalação e Importação das Bibliotecas

Nesta etapa iremos importar as bibliotecas e pacotes necessários para fazer todo o projeto. Conforme visto no módulo de introdução à Python, existem diversas bibliotecas prontas para facilitar o trabalho de análise de dados, e iremos utilizar algumas delas aqui.

In [38]:
# Instalação das bibliotecas
!pip install -q lifelines

In [39]:
# Bibliotecas básicas
import pandas as pd
pd.options.mode.chained_assignment = None  # Removendo warnings3
import warnings
warnings.filterwarnings('ignore') # Removendo warnings
import numpy as np
import datetime as dt
import statistics as st
import itertools
from scipy import stats

# Bibliotecas de visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Importação das bibliotecas de predição
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, r2_score, mean_squared_error, precision_score, recall_score, max_error
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import export_graphviz, DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier

# Imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Kaplan-Meier curve
from lifelines import KaplanMeierFitter

# Bibliotecas adicionais
from dateutil.relativedelta import relativedelta
import graphviz
from sklearn.tree import export_graphviz
from io import StringIO
from IPython.display import Image  
import pydotplus

### 2) Conexão com a Base de Dados

Iremos importar diretamente para nosso código a base de dados dos colaboradores. Ela está localizada em uma pasta no Google Drive, e é possível fazer essa conexão diretamente.

In [40]:
# Pegando o arquivo csv no diretório
path = 'https://raw.githubusercontent.com/higor-gomes93/employee_attrition/main/employee_attrition.csv'
dataset_inicial = pd.read_csv(path)

In [41]:
# Conferindo a conexão
dataset_inicial

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


### 3) Observando os Dados

Nessa etapa iremos analisar previamente os dados, observando quais atributos temos disponíveis, o total de observações, algumas medidas estatísticas, tratamentos a serem feitos, o formato e o tipo dos dados, entre outras.

#### 3.1 Observações iniciais

In [42]:
# Observando o formato do dataset inicial
dataset_inicial.head(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0


In [43]:
# Observando o total de linhas e colunas
dataset_inicial.shape

(1470, 35)

In [44]:
# Observando o nome das colunas
dataset_inicial.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [45]:
# Observando algumas medidas estatísticas do dataset inicial
dataset_inicial.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [46]:
# Observando todas as medidas estatísticas do dataset inicial
dataset_inicial.describe(include = 'all')

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470,1470,1470.000000,1470,1470.000000,1470.000000,1470,1470.0,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
unique,NaN,2,3,NaN,3,NaN,NaN,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,No,Travel_Rarely,NaN,Research & Development,NaN,NaN,Life Sciences,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1233,1043,NaN,961,NaN,NaN,606,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,36.923810,NaN,NaN,802.485714,NaN,9.192517,2.912925,NaN,1.0,1024.865306,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,NaN,NaN,403.509100,NaN,8.106864,1.024165,NaN,0.0,602.024335,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,NaN,NaN,102.000000,NaN,1.000000,1.000000,NaN,1.0,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,NaN,NaN,465.000000,NaN,2.000000,2.000000,NaN,1.0,491.250000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,NaN,NaN,802.000000,NaN,7.000000,3.000000,NaN,1.0,1020.500000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,NaN,NaN,1157.000000,NaN,14.000000,4.000000,NaN,1.0,1555.750000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000


In [47]:
# Algumas informações adicionais
dataset_inicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

#### 3.2) Observando valores nulos

In [48]:
# Identificando valores nulos
dataset_inicial.isnull().any()

Age                         False
Attrition                   False
BusinessTravel              False
DailyRate                   False
Department                  False
DistanceFromHome            False
Education                   False
EducationField              False
EmployeeCount               False
EmployeeNumber              False
EnvironmentSatisfaction     False
Gender                      False
HourlyRate                  False
JobInvolvement              False
JobLevel                    False
JobRole                     False
JobSatisfaction             False
MaritalStatus               False
MonthlyIncome               False
MonthlyRate                 False
NumCompaniesWorked          False
Over18                      False
OverTime                    False
PercentSalaryHike           False
PerformanceRating           False
RelationshipSatisfaction    False
StandardHours               False
StockOptionLevel            False
TotalWorkingYears           False
TrainingTimesL

#### 3.3) Observando outliers

In [49]:
# Criando dicionários para adicioinar os valores
dict_outliers_zscore = {}
dict_outliers_iqr = {}
dict_outliers_percentile = {}

In [50]:
# Definindo uma função para coleta de outliers por meio do método Z-score com maior robustez
def z_r_score_outlier(data):
    outliers = []
    med = np.median(data)
    ma = stats.median_absolute_deviation(data)
    for i in data: 
        z = (0.6745*(i-med))/ (np.median(ma))
        if np.abs(z) > 3: 
            outliers.append(i)
    return outliers

# Definindo uma função para coleta de outliers por meio do método IQR
def iqr_outliers(data):
    outliers = []
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3-q1
    Lower_tail = q1 - 2.5 * iqr
    Upper_tail = q3 + 2.5 * iqr
    for i in data:
        if i > Upper_tail or i < Lower_tail:
            outliers.append(i)
    return outliers

# Definindo uma função para coleta de outliers por meio da análise do percentil
def winsorization_outliers(data):
    outliers = []
    q1 = np.percentile(data, 1)
    q3 = np.percentile(data, 99)
    for i in data:
        if i > q3 or i < q1:
            outliers.append(i)
    return outliers

In [51]:
# Coletando os outliers
for i in dataset_inicial.columns:
    if dataset_inicial[i].dtype == 'int64':
        outliers_zscore = z_r_score_outlier(dataset_inicial[i])
        if len(outliers_zscore) > 0:
            dict_outliers_zscore[i] = outliers_zscore
        outliers_iqr = iqr_outliers(dataset_inicial[i])
        if len(outliers_iqr) > 0:
            dict_outliers_iqr[i] = outliers_iqr
        outliers_percentile = winsorization_outliers(dataset_inicial[i])
        if len(outliers_percentile) > 0:
            dict_outliers_percentile[i] = outliers_percentile

### 4) Limpeza e Tratamento dos Dados

Etapa de remoção de outliers, padronização de categorias, tratamento de valores faltantes e outras transformações necessárias.

#### 4.1) Removendo colunas sem valor semântico

In [52]:
# Colunas sem importância para as análises
dataset_inicial = dataset_inicial.drop(columns = ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'])

#### 4.2) Valores faltantes

In [53]:
# Conforme indicado anteriormente, não existem valores nulos na base de dados
print('Não existem valores nulos.') if any(list(dataset_inicial.isnull().any())) == False else print('Existem valores nulos.')

Não existem valores nulos.


#### 4.3) Outliers e valores fora do padrão

In [54]:
# Iremos nos basear na técnica que analise o percentil dos dados
# Coletando os outliers
dict_outliers_percentile = {}
for i in dataset_inicial.columns:
    if dataset_inicial[i].dtype == 'int64':
        outliers_percentile = winsorization_outliers(dataset_inicial[i])
        if len(outliers_percentile) > 0:
            dict_outliers_percentile[i] = outliers_percentile

# Observando os outliers
for i in dict_outliers_percentile.items():
    print(i)

('Age', [59, 59, 59, 59, 59, 59, 18, 18, 60, 60, 18, 60, 18, 59, 59, 18, 60, 59, 59, 18, 18, 60, 18])
('DailyRate', [103, 1488, 111, 1496, 111, 106, 1490, 1490, 1499, 1495, 102, 109, 1492, 111, 116, 107, 1498, 1495, 1490, 1496, 115, 104, 1495, 1490, 116, 105])
('MonthlyIncome', [1232, 19926, 1102, 19999, 1200, 1009, 1281, 19859, 1051, 19973, 19845, 1052, 19627, 19943, 19740, 1223, 1118, 19847, 19717, 19701, 1359, 1261, 1274, 19658, 19833, 19665, 1081, 1091, 19636, 1129])
('MonthlyRate', [2094, 26959, 26897, 26820, 2302, 2137, 26767, 26707, 26914, 2227, 2288, 2112, 2125, 26894, 26999, 2104, 2243, 26968, 2253, 26933, 2323, 2261, 2097, 26997, 26841, 2125, 2122, 26862, 26849, 26956])
('TotalWorkingYears', [0, 37, 38, 40, 0, 36, 37, 0, 0, 36, 37, 0, 40, 0, 0, 0, 36, 36, 0, 36, 0, 36, 37, 0])
('YearsAtCompany', [37, 40, 33, 33, 36, 32, 34, 32, 33, 33, 32, 33, 36])
('YearsInCurrentRole', [16, 18, 18, 17, 16, 16, 16, 16, 16, 17, 17, 17, 16])
('YearsSinceLastPromotion', [15, 15, 15, 15, 15, 15,